<a href="https://colab.research.google.com/github/hammaad2002/ASRAdversarialAttacks/blob/deepspeech/Tutorial(How_to_use_this_repository).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>**Disclaimer:**
#Work in progress !!!
Till now only FGSM and BIM attacks work for deepspeech model.

In [ ]:
%%capture
!git clone -b deepspeech https://github.com/hammaad2002/ASRAdversarialAttacks.git
%cd /content/ASRAdversarialAttacks
!python install_dependencies.py
%cd /content
!git clone https://github.com/hammaad2002/CRDNN_Model.git
!git clone https://github.com/SeanNaren/deepspeech.pytorch.git && cd deepspeech.pytorch && python setup.py install
%cd /content/deepspeech.pytorch

In [ ]:
import torch
import torchaudio
import numpy as np
import warnings
from IPython.display import Audio

warnings.filterwarnings("ignore")

CODE:

-----------------------

In [207]:
from typing import List, Tuple
from tqdm.notebook import tqdm
from scipy import signal
import librosa
import numpy as np
import torch
import scipy
import wget
from functools import reduce
from pkg_resources import packaging
import pytorch_lightning as pl
from deepspeech_pytorch.loader.data_loader import _collate_fn
from deepspeech_pytorch.configs.inference_config import LMConfig
from deepspeech_pytorch.enums import DecoderType
from deepspeech_pytorch.configs.inference_config import LMConfig
from deepspeech_pytorch.utils import load_decoder, load_model
import warnings

warnings.filterwarnings("ignore")

class ASRAttacks(object):
    '''
    Adversarial Attack on ASR model. Right now it is specifically implemented for
    wav2vec2.0 model from torchaudio hub.

    It support the following attacks:
    1) Fast Gradient Sign Method  (FGSM)
    2) Basic Iterative Moment     (BIM)
    3) Projected Gradient Descent (PGD)
    4) Carlini and Wagner Attack  (CW)
    5) Imperceptible ASR Attack   (I-ASR)
    '''
    def __init__(self, decoder_type):
        '''
        Creates an instance of the class "ASRAttacks".

        INPUT ARGUMENTS:

        model  : The model on which the attack is supposed to be performed.
        device : Either 'cpu' if we have only CPU or 'cuda' if we have GPU
        NEED TO FIX ALL DOC THE STRINGS
        '''

        filename = "librispeech_pretrained_v3.ckpt"
        model_url = "https://github.com/SeanNaren/deepspeech.pytorch/releases/download/V3.0/librispeech_pretrained_v3.ckpt"
        model_path = '/content/' + filename
        wget.download(model_url, model_path)
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model = load_model(device=self.device, model_path=model_path)
        self.criterion = self.model.criterion
        self.decoder_type = decoder_type
        lm_config = LMConfig()
        if self.decoder_type == "greedy":
          lm_config.decoder_type = DecoderType.greedy
        elif decoder_type == "beam":
          lm_config.decoder_type = DecoderType.beam
        else:
          raise ValueError(f"Decoder type {decoder_type} currently not supported.")
        lm_config.lm_path = ""
        lm_config.top_paths = 1
        lm_config.alpha = 0.0
        lm_config.beta = 0.0
        lm_config.cutoff_top_n = 40
        lm_config.cutoff_prob = 1.0
        lm_config.beam_width = 10
        lm_config.lm_workers = 4
        lm_config = lm_config
        self.decoder = load_decoder(labels=self.model.labels, cfg=lm_config)

    def _transform_model_input(
        self, x, y = None, compute_gradient = False, tensor_input = False, real_lengths = None):

        # extracting stft information from the given config
        window_name = self.model.spect_cfg["window"].value
        sample_rate = self.model.spect_cfg["sample_rate"]
        window_size = self.model.spect_cfg["window_size"]
        window_stride = self.model.spect_cfg["window_stride"]

        # calculating stft arguments
        n_fft = int(sample_rate * window_size)
        hop_length = int(sample_rate * window_stride)
        win_length = n_fft

        # Get window for the transformation
        if window_name == "hamming":
            window_fn = torch.hamming_window
        elif window_name == "hann":
            window_fn = torch.hann_window
        elif window_name == "blackman":
            window_fn = torch.blackman_window
        elif window_name == "bartlett":
            window_fn = torch.bartlett_window
        else:
            raise NotImplementedError(f"Spectrogram window {window_name} not supported.")

        # Create a transformer to transform between the two spaces
        transformer = torchaudio.transforms.Spectrogram(
            n_fft=n_fft, hop_length=hop_length, win_length=win_length, window_fn=window_fn, power=None
        )
        transformer.to(self.device)

        # Create a label map
        label_map = {self.model.labels[i]: i for i in range(len(self.model.labels))}

        # We must process each sequence separately due to the diversity of their length
        batch = []
        for i, _ in enumerate(x):
            # First process the target
            if y is None:
                target = []
            else:
                target = list(filter(None, [label_map.get(letter) for letter in list(y[i])]))

            # Push the sequence to device
            if isinstance(x, np.ndarray) and not tensor_input:
                x[i] = x[i].astype(np.float32)
                x[i] = torch.tensor(x[i]).to(self.device)

            # Set gradient computation permission
            if compute_gradient:
                x[i].requires_grad = True

            # Transform the sequence into the frequency space
            if tensor_input and real_lengths is not None:
                transformed_input = transformer(x[i][: real_lengths[i]])
            else:
                transformed_input = transformer(x[i])

            if packaging.version.parse(torch.__version__) >= packaging.version.parse("1.10.0"):
                spectrogram = torch.abs(transformed_input)
            else:
                spectrogram, _ = torchaudio.functional.magphase(transformed_input)
            spectrogram = torch.log1p(spectrogram)

            # Normalize data
            mean = spectrogram.mean()
            std = spectrogram.std()
            spectrogram = spectrogram - mean
            spectrogram = spectrogram / std

            # Then form the batch
            batch.append((spectrogram, target))

        # We must keep the order of the batch for later use as the following function will change its order
        batch_idx = sorted(range(len(batch)), key=lambda i: batch[i][0].size(1), reverse=True)

        # The collate function is important to convert input into model space
        inputs, targets, input_percentages, target_sizes = _collate_fn(batch)

        return inputs, targets, input_percentages, target_sizes, batch_idx

    def FGSM_ATTACK(self, input__: torch.Tensor, target: List[str]= None,
           epsilon: float = 0.2, targeted: bool = False) -> np.ndarray:

        '''
        Simple fast gradient sign method attack is implemented which is the simplest
        adversarial attack in this domain.
        For more information, see the paper: https://arxiv.org/pdf/1412.6572.pdf

        INPUT ARGUMENTS:

        input__       : Input audio. Ex: Tensor[0.1,0.3,...] or (samples,)
                        Type: torch.Tensor

        target        : Target transcription (needed if the you want targeted
                        attack) Ex: ["my name is mango."].
                        Type: List[str]
                        CAUTION:
                        Please make sure these characters are also present in the
                        dictionary of the model also.

        epsilon       : Noise controlling parameter or step size.
                        Type: float

        targeted      : If the attack should be targeted towards your desired
                        transcription or not.
                        Type: bool
                        CAUTION:
                        Please make to pass your targetted
                        transcription also in this case).

        RETURNS:

        np.ndarray : Perturbed audio
        '''
        # Cloning the original audio
        input_ = input__.clone()

        # Putting the model in training mode so that backpropagation can be done
        # But also need to disable batchnorm in this process
        self.model.train()
        self.set_batchnorm(train=False)

        # Making our input differentiable
        input_.requires_grad = True

        # Transform data into the model input space
        input, _, input_rate, _, _ = self._transform_model_input(x=[input_.squeeze().to(self.device)], compute_gradient=False)

        # Compute real input sizes
        input_size = input_rate.mul_(input.size()[-1]).int()

        # Forward Pass
        output, output_size, *_ = self.model(input.to(self.device), input_size.to(self.device))

        # Softmax Activation for computing logits
        output = output.transpose(0, 1)
        output = output.log_softmax(-1)

        if targeted: # Condition for checking if the user wants 'targeted' attack to be performed

            # Assert that in targeted attack we have target present before we proceed
            assert target != None, "Target should not be 'None' in targeted attack. Please pass a target transcription."

            # Encode the target transcription
            _, target, _, target_size, _ = self._transform_model_input(x=[input_.squeeze().to(self.device)], y=[target], compute_gradient=False)
            target = target.to(self.device).long()

            # Computing the CTC Loss
            loss = self.criterion(output, target, output_size, target_size).to(self.device)

            # Computing gradient of our input w.r.t loss
            loss.backward()

            # If 'targeted' then we will minimize our loss to the respective target transcription
            sign_grad = -input_.grad.sign()

            # Calculating 'sign' of the FGSM attack and multiplying it with our small epsilon step
            perturbation = epsilon * sign_grad

            # Adding perturbation in the original input to make adversarial example
            perturbed_input = input_ + perturbation

            # Clamping the audio in original audio range (-1,1)
            perturbed_input = torch.clamp(perturbed_input, -1, 1)

            # Returning perturbed audio
            self.set_batchnorm(train=True)
            return perturbed_input.detach().numpy()

        else: # Condition for checking if the user wants 'untargeted' attack to be performed

            # Using the model's transcription as target in untargeted attack
            untarget = list(self.INFER(input_.to(self.device)))

            # Encode the target transcription
            _, target, _, target_size, _ = self._transform_model_input(x=[input_.squeeze().to(self.device)], y=[untarget], compute_gradient=False)
            target = target.to(self.device).long()

            # Computing the CTC Loss
            loss = self.criterion(output, target, output_size, target_size).to(self.device)

            # Computing gradient of our input w.r.t loss
            loss.backward()

            # If untargeted then we will maximize our loss
            sign_grad = input_.grad.sign()

            # Calculating 'sign' of the FGSM attack and multiplying it with our small epsilon step
            perturbation = epsilon * sign_grad

            # Adding perturbation in the original input to make adversarial example
            perturbed_input = input_ + perturbation

            # Clamping the audio in original audio range (-1,1)
            perturbed_input = torch.clamp(perturbed_input, -1, 1)

            # Returning perturbed audio
            self.set_batchnorm(train=True)
            return perturbed_input.detach().numpy()

    def BIM_ATTACK(self, input__: torch.Tensor, target: List[str] = None,
          epsilon: float = 0.2, alpha: float = 0.1,
          num_iter: int = 10, nested: bool = True,targeted: bool = False, early_stop: bool = False) -> np.ndarray:

        '''
        Basic Itertive Moment attack is implemented which is simple Fast Gradient
        Sign Attack but in loop.
        For more information, see the paper: https://arxiv.org/abs/1607.02533

        INPUT ARGUMENTS:

        input__       : Input audio. Ex: Tensor[0.1,0.3,...] or (samples,)
                        Type: torch.Tensor

        target        : Target transcription (needed if the you want targeted
                        attack) Ex: ["my name is mango."].
                        Type: List[str]
                        CAUTION:
                        Please make sure these characters are also present in the
                        dictionary of the model also.

        epsilon       : Maximum allowable noise for our audio.
                        Type: float

        alpha         : Step size for noise to be added in each iteration
                        Type: float

        num_iter      : Number of iteration of attack
                        Type: int

        nested        : if this attack in being run in a for loop with tqdm
                        Type: bool

        targeted      : If the attack should be targeted towards your desired
                        transcription or not.
                        Type: bool
                        CAUTION:
                        Please make to pass your targetted
                        transcription also in this case).

        early_stop    : If user wants to stop the attack early if the attack reaches the target transcription before the total number of iterations.
                        Type: bool

        RETURNS:

        np.ndarray : Perturbed audio
        '''

        # Cloning the original given audio
        input_ = input__.clone()

        # Making our input differentiable
        input_.requires_grad = True

        # Storing input in variable to add in noise later
        original_input = input_.clone()

        # Checking if the user is running this code in for loop or not
        if nested:
            leave = False

        else:
            leave = True

        if targeted: # Condition for checking if the user wants 'targeted' attack to be performed

            # Assert that in targeted attack we have target present before we proceed
            assert target != None, "Target should not be 'None' in targeted attack. Please pass a target transcription."

            # Encode the target transcription
            input, target__, input_rate, target_size, _= self._transform_model_input(x=[input_.squeeze().to(self.device)], y =[target], compute_gradient=False)

            # Compute real input sizes
            input_size = input_rate.mul_(input.size()[-1]).int()

            for i in tqdm(range(num_iter), colour="red", leave = leave):

                # Putting the model in training mode so that backpropagation can be done
                # But also need to disable batchnorm in this process
                self.model.train()
                self.set_batchnorm(train=False)

                # Forward pass
                output, output_size, *_ = self.model(input.to(self.device), input_size.to(self.device))

                # Convert the target transcription to a PyTorch tensor
                target_ = target__.to(self.device).long()

                # Computing the CTC Loss
                output = output.transpose(0, 1)
                output = output.log_softmax(-1)
                loss = self.criterion(output, target_, output_size, target_size).to(self.device)

                # Computing gradients of our input w.r.t loss
                loss.backward(retain_graph=True)

                # If targeted then we will minimize our loss
                sign_grad = -input_.grad.data

                # Adding perturbation in our input
                perturbed_input = original_input + (alpha * sign_grad.detach().sign())

                # Clamping the perturbation in range (-eps, eps)
                perturbation = torch.clamp(perturbed_input - original_input, -epsilon, epsilon)

                # Clamping the overall perturbated audio in the original audio range (-1, 1)
                input_.data = torch.clamp(input_ + perturbation, -1, 1)

                if early_stop: # if user have enabled early stopping then do the following tasks or else run all iterations

                    # Storing model's current inference and target transcription in new variables for computing WER
                    string1 = list(filter(lambda x: x!= '',self.INFER(input_).split(" ")))
                    string2 = target.split(" ")

                    # Computing WER while also making sure length of both strings is same
                    # This will also early stop the attack if we reach out target transcription
                    # before the completion of all iterations because further iterations will further
                    # increase noise in the original audio leading to bad/low SNR
                    if len(string1) == len(string2):
                        if self._wer(string1, string2)[0] == 0:
                            print("Breaking for loop because targeted Attack is performed successfully !")
                            return input_.detach().numpy()
                    elif len(string1) > len(string2):
                        diff = len(string1) - len(string2)
                        for i in range(diff):
                            string2.append("<eps>")
                        if self._wer(string1, string2)[0] == 0:
                            print("Breaking for loop because targeted Attack is performed successfully !")
                            return input_.detach().numpy()
                    else:
                        diff = len(string2) - len(string1)
                        for i in range(diff):
                            string1.append("<eps>")
                        if self._wer(string1, string2)[0] == 0:
                            print("Breaking for loop because targeted Attack is performed successfully !")
                            return input_.detach().numpy()

                # Making gradients of input zero
                input_.grad.zero_()

            # Returning perturbed audio after the loop ends
            self.set_batchnorm(train=True)
            return input_.detach().numpy()

        else: # Condition for checking if the user wants 'untargeted' attack to be performed

            for i in tqdm(range(num_iter), colour="red", leave = leave):

                # Putting the model in training mode so that backpropagation can be done
                # But also need to disable batchnorm in this process
                self.model.train()
                self.set_batchnorm(train=False)

                # Using the model's transcription as target in untargeted attack
                untarget = self.INFER(input_.to(self.device))

                # Encode the target transcription
                input, target__, input_rate, target_size, _= self._transform_model_input(x=[input_.squeeze().to(self.device)], y =[untarget], compute_gradient=False)

                # Compute real input sizes
                input_size = input_rate.mul_(input.size()[-1]).int()

                # Forward pass
                output, output_size, *_ = self.model(input.to(self.device), input_size.to(self.device))

                # Convert the target transcription to a PyTorch tensor
                target_ = target__.to(self.device).long()

                # Computing the CTC Loss
                output = output.transpose(0, 1)
                loss = self.criterion(output, target_, output_size, target_size).to(self.device)

                # Computing gradients of our input w.r.t loss
                loss.backward(retain_graph=True)

                # If untargeted then we will maximize our loss
                sign_grad = input_.grad.data

                # Adding perturbation in our input
                perturbed_input = original_input + (alpha * sign_grad.detach().sign())

                # Clamping the perturbation in range (-eps, eps)
                perturbation = torch.clamp(perturbed_input - original_input, -epsilon, epsilon)

                # Clamping the overall perturbated audio in the original audio range (-1, 1)
                input_.data = torch.clamp(input_ + perturbation, -1, 1)

                if early_stop: # if user have enabled early stopping then do the following tasks or else run all iterations

                    # Storing model's current inference and target transcription in new variables for computing WER
                    string1 = list(self.INFER(input_).split(" "))
                    string2 = list(untarget.split(" "))

                    # Removing empty spaces (if any) that cause error when computing WER
                    string1 = list(filter(lambda x: x!= '', string1))
                    string2 = list(filter(lambda x: x!= '', string2))

                    # Computing WER while also making sure length of both strings is same
                    # This will also early stop the attack if we reach out target transcription
                    # before the completion of all iteration because further iteration will further
                    # increase noise in the original audio leading to bad/low SNR
                    if len(string1) == len(string2):
                        if self._wer(string1, string2)[0] == 1:
                            print("Breaking for loop because untargeted Attack is performed successfully !")
                            return input_.detach().numpy()
                    elif len(string1) > len(string2):
                        diff = len(string1) - len(string2)
                        for i in range(diff):
                            string2.append("<eps>")
                        if self._wer(string1, string2)[0] == 1:
                            print("Breaking for loop because untargeted Attack is performed successfully !")
                            return input_.detach().numpy()
                    else:
                        diff = len(string2) - len(string1)
                        for i in range(diff):
                            string1.append("<eps>")
                        if self._wer(string1, string2)[0] == 1:
                            print("Breaking for loop because untargeted Attack is performed successfully !")
                            return input_.detach().numpy()

                # Making gradients of input zero
                input_.grad.zero_()

            # Returning perturbed audio after the loop ends
            self.set_batchnorm(train=True)
            return input_.detach().numpy()

    def PGD_ATTACK(self, input__: torch.Tensor, target: List[str] = None,
                 epsilon: float = 0.3, alpha: float = 0.01, num_iter: int = 40,
                 nested: bool = True,targeted: bool = False, early_stop: bool = False) -> np.ndarray:

        '''
        Projected Gradient Descent attack is implemented which in simple terms is more
        advanced version of BIM. In this attack we project our perturbation back to
        some Lp norm and find perturbations in that particular region.
        For more information, see the paper: https://arxiv.org/abs/1706.06083

        INPUT ARGUMENTS:

        input__       : Input audio. Ex: Tensor[0.1,0.3,...] or (samples,)
                        Type: torch.Tensor

        target        : Target transcription (needed if the you want targeted
                        attack) Ex: ["my name is mango."].
                        Type: List[str]
                        CAUTION:
                        Please make sure these characters are also present in the
                        dictionary of the model also.

        epsilon       : Noise controlling parameter.
                        Type: float

        alpha         : Step size for noise to be added in each iteration
                        Type: float

        num_iter      : Number of iteration of attack
                        Type: int

        nested        : if this attack in being run in a for loop with tqdm
                        Type: bool

        targeted      : If the attack should be targeted towards your desired
                        transcription or not.
                        Type: bool
                        CAUTION:
                        Please make to pass your targetted
                        transcription also in this case).

        early_stop    : If user wants to stop the attack early if the attack reaches the target transcription before the total number of iterations.
                        Type: bool

        RETURNS:

        np.ndarray : Perturbed audio
        '''

        # Cloning the original audio
        input_ = input__.clone().to(self.device)

        # Making a zero differentiable tensor of same shape as input
        delta = (torch.rand_like(input_, requires_grad=True) - 0.5) * 2 * 1e-9 # tensor with value in between (-1e-9, 1e-9) almost ≈ 0

        # checking if the user is running this code in for loop or not
        if nested:
            leave = False

        else:
            leave = True

        if targeted: # Condition for checking if the user wants 'targeted' attack to be performed

            # Assert that in targeted attack we have target present before we proceed
            assert target != None, "Target should not be 'None' in targeted attack. Please pass a target transcription."

            # Encode the target transcription
            input, target__, input_rate, target_size, _= self._transform_model_input(x=[input_.squeeze().to(self.device)], y =[target], compute_gradient=False)

            # Compute real input sizes
            input_size = input_rate.mul_(input.size()[-1]).int()

            for i in tqdm(range(num_iter), colour = 'red', leave = leave):

                # Putting the model in training mode so that backpropagation can be done
                # But also need to disable batchnorm in this process
                self.model.train()
                self.set_batchnorm(train=False)

                # Delta's input transformation
                delta_, _, _, _, _= self._transform_model_input(x=[delta.squeeze().to(self.device)], compute_gradient=False)

                # Forward pass
                print("Input shape:", input.shape)
                print("Max input:", max(input))
                print("Delta shape:", delta_.shape)
                print("Max delta:", max(delta_))
                new_input = input.to(self.device) + delta_.to(self.device)
                print("New input shape:", new_input.shape)
                print("Max new input:", max(new_input))
                output, output_size, *_ = self.model(new_input, input_size.to(self.device))
                # Check the grad_fn attribute.
                if delta.grad_fn is not None:
                    print("delta is differentiable")
                else:
                    print("delta is not differentiable")

                # Convert the target transcription to a PyTorch tensor
                target_ = target__.to(self.device).long()

                # Computing the CTC Loss
                output = output.transpose(0, 1)
                output = output.log_softmax(-1)
                print("Output:", output)
                print("Target:", target_)
                print("Output size:", output_size)
                print("Target size:", target_size)
                loss = self.criterion(output, target_, output_size, target_size).to(self.device)
                print(loss)

                # Computing gradients of our input w.r.t loss
                loss.backward(retain_graph = True)

                # Update delta with gradient sign
                print(delta.grad)
                print(delta.grad())
                sign = -1 # Negative sign because of targeted attack
                delta.data = (delta + alpha * sign * delta.grad.detach().sign())

                # Perform projection of delta onto Lp ball
                delta.data = delta.data.clamp(-epsilon, epsilon)

                if early_stop: # if user have enabled early stopping then do the following tasks or else run all iterations

                    # Storing model's current inference and target transcription in new variables for computing WER
                    string1 = list(filter(lambda x: x!= '',self.INFER(input_).split(" ")))
                    string2 = target.split(" ")

                    # Computing WER while also making sure length of both strings is same
                    # This will also early stop the attack if we reach out target transcription
                    # before the completion of all iteration because further iteration will further
                    # increase noise in the original audio leading to bad/low SNR
                    if len(string1) == len(string2):
                        if self._wer(string1, string2)[0] == 0:
                            print("Breaking for loop because targeted Attack is performed successfully !")
                            adv_example = input_ + delta
                            return adv_example.detach().cpu().numpy()
                    elif len(string1) > len(string2):
                        diff = len(string1) - len(string2)
                        for i in range(diff):
                            string2.append("<eps>")
                        if self._wer(string1, string2)[0] == 0:
                            print("Breaking for loop because targeted Attack is performed successfully !")
                            adv_example = input_ + delta
                            return adv_example.detach().cpu().numpy()
                    else:
                        diff = len(string2) - len(string1)
                        for i in range(diff):
                            string1.append("<eps>")
                        if self._wer(string1, string2)[0] == 0:
                            print("Breaking for loop because targeted Attack is performed successfully !")
                            adv_example = input_ + delta
                            return adv_example.detach().cpu().numpy()

                # Zeroing the gradients so that they don't accumulate
                delta.grad.zero_()

            # Returning perturbed audio after the loop ends
            adv_example = input_ + delta
            self.set_batchnorm(train=True)
            return adv_example.detach().cpu().numpy()

        else: # Condition for checking if the user wants 'untargeted' attack to be performed

            # Encode the target transcription
            input, _, input_rate, _, _= self._transform_model_input(x=[input_.squeeze().to(self.device)], compute_gradient=False)

            # Compute real input sizes
            input_size = input_rate.mul_(input.size()[-1]).int()

            for i in tqdm(range(num_iter), colour = 'red', leave = leave):

                # Putting the model in training mode so that backpropagation can be done
                # But also need to disable batchnorm in this process
                self.model.train()
                self.set_batchnorm(train=False)

                # Using the model's transcription as target in untargeted attack
                untarget = self.INFER(input_.to(self.device))

                # Encode the target transcription
                delta_, target__, _, target_size, _= self._transform_model_input(x=[delta.squeeze().to(self.device)], y =[untarget], compute_gradient=False)

                # Forward pass
                output, output_size, *_ = self.model(input.to(self.device) + delta_.to(self.device), input_size.to(self.device))

                # Convert the target transcription to a PyTorch tensor
                target_ = target__.to(self.device).long()

                # Computing the CTC Loss
                output = output.transpose(0, 1)
                output = output.log_softmax(-1)
                loss = self.criterion(output, target_, output_size, target_size).to(self.device)

                # Computing gradients of our input w.r.t loss
                loss.backward(retain_graph = True)

                # Update delta with gradient sign
                sign = 1 # Positive sign because of untargeted attack
                delta.data = (delta + alpha * sign * delta.grad.detach().sign())

                # Perform projection of delta onto Lp ball
                delta.data = delta.data.clamp(-epsilon, epsilon)

                if early_stop: # if user have enabled early stopping then do the following tasks or else run all iterations

                    # Storing model's current inference and target transcription in new variables for computing WER
                    string1 = list(self.INFER(input_).split(" "))
                    string2 = list(untarget.split(" "))

                    # Computing WER while also making sure length of both strings is same
                    # This will also early stop the attack if we reach out target transcription
                    # before the completion of all iteration because further iteration will further
                    # increase noise in the original audio leading to bad/low SNR
                    if len(string1) == len(string2):
                        if self._wer(string1, string2)[0] == 1:
                            print("Breaking for loop because untargeted Attack is performed successfully !")
                            adv_example = input_ + delta
                            return adv_example.detach().cpu().numpy()
                    elif len(string1) > len(string2):
                        diff = len(string1) - len(string2)
                        for i in range(diff):
                            string2.append("<eps>")
                        if self._wer(string1, string2)[0] == 1:
                            print("Breaking for loop because untargeted Attack is performed successfully !")
                            adv_example = input_ + delta
                            return adv_example.detach().cpu().numpy()
                    else:
                        diff = len(string2) - len(string1)
                        for i in range(diff):
                            string1.append("<eps>")
                        if self._wer(string1, string2)[0] == 1:
                            print("Breaking for loop because untargeted Attack is performed successfully !")
                            adv_example = input_ + delta
                            return adv_example.detach().cpu().numpy()

                # Zeroing the gradients so that they don't accumulate
                delta.grad.zero_()

            # Returning perturbed audio after the loop ends
            adv_example = input_ + delta
            self.set_batchnorm(train=True)
            return adv_example.detach().cpu().numpy()

    def CW_ATTACK(self, input__: torch.Tensor, target: List[str] = None,
           epsilon: float = 0.3, c: float = 1e-4, learning_rate: float = 0.01,
           num_iter: int = 1000, decrease_factor_eps: float = 1,
           num_iter_decrease_eps: int = 10, optimizer: str = None,
           nested: bool = True, early_stop: bool = True, search_eps: bool = False,
           targeted: bool = False, internal_call = False) -> np.ndarray:

        '''
        Implements the Carlini and Wagner attack, the strongest white box
        adversarial attack. This attack uses an optimization strategy to find the
        adversarial transcription while keeping the perturbation as low as possible.
        For more information, see the paper: https://arxiv.org/pdf/1801.01944.pdf

        INPUT ARGUMENTS:

        input__       : Input audio. Ex: Tensor[0.1,0.3,...] or (samples,)
                        Type: torch.Tensor

        target        : Target transcription (needed if the you want targeted
                        attack) Ex: ["my name is mango."].
                        Type: List[str]
                        CAUTION:
                        Please make sure these characters are also present in the
                        dictionary of the model also.

        epsilon       : Noise controlling parameter.
                        Type: float

        c             : Regularization term controlling factor.
                        Type: float

        learning_rate : learning_rate of optimizer.
                        Type: float

        num_iter      : Number of iteration of attack.
                        Type: int

        decrease_factor_eps   : Factor to decrease epsilon during search
                                Type: float

        num_iter_decrease_eps : Number of iterations after which to decrease epsilon
                                Type: int

        optimizer     : Name of the optimizer to use for the attack.
                        Type: str

        nested        : if this attack in being run in a for loop with tqdm
                        Type: bool

        early_stop    : if the user wants to end the attack as soon as the attack
                        gets the target transcription.
                        Type: bool

        search_eps    : if the user wants the attack to search for the epsilon value
                        on its own provided the initial epsilon value of large.
                        Type: bool

        targeted      : If the attack should be targeted towards your desired
                        transcription or not.
                        Type: bool
                        CAUTION:
                        Please make to pass your targetted
                        transcription also in this case).

        internal_call : If the CW is being called internally by another attack.
                        Type: bool

        RETURNS:

        np.ndarray : Perturbed audio
        '''

        # checking if user accidentally passed wrong arugments or not
        if early_stop == True and search_eps == True:
            raise Exception("Early stop and Epsilon search arguments, both cannot be true at the same time.")

        if epsilon <= 0:
            raise Exception("Value of epsilon should be greater than 0")

        # Convert the input audio to a PyTorch tensor
        input_audio = input__.clone().to(self.device).float()

        # Making audio differentiable
        input_audio.requires_grad_()

        # Cloning the original audio
        input_audio_orig = input_audio.clone().to(self.device)

        # Define the optimizer
        if optimizer == "Adam":

            optimizer = torch.optim.Adam([input_audio], lr=learning_rate)

        else:

            optimizer = torch.optim.SGD([input_audio], lr=learning_rate)

        # Setting our inital parameters
        successful_attack = False
        num_successful_attacks = 0

        # Checking if the user wants to run this code in for loop or not
        if nested:
            leave = False
            descrip = None
            if internal_call:
              descrip = "*"*5+"Attack Stage 1"+"*"*5

        else:
            leave = True
            descrip = None

        if targeted:

            # Making sure target is given in targeted attack
            assert target is not None, "Target should not be 'None' in a targeted attack. Please pass a target transcription."

            # Encode the target transcription
            encoded_transcription = self._encode_transcription(target)

            # Convert the target transcription to a PyTorch tensor
            target_tensor = torch.from_numpy(np.array(encoded_transcription)).to(self.device).long()

            for i in tqdm(range(num_iter), colour="red", leave = leave, desc = descrip):

                # Zero the gradients
                optimizer.zero_grad()

                # Compute the model’s prediction
                output, _ = self.model(input_audio)

                # Softmax Activation for computing logits
                output = F.log_softmax(output, dim=-1)

                # Compute the CTC loss function
                output_length = torch.tensor([output.shape[1]], dtype=torch.long).to(self.device)
                output = output.transpose(0, 1)
                target_length = torch.tensor([len(encoded_transcription)], dtype=torch.long).to(self.device)
                loss_classifier = F.ctc_loss(output, target_tensor, output_length, target_length, blank=0, reduction='mean')

                # Regularization term to minimize the perturbation
                loss_norm = torch.norm(input_audio - input_audio_orig)

                # Combine the losses and compute gradients
                loss = (c * loss_norm) + ( loss_classifier)

                # Computing gradients of our input w.r.t loss
                loss.backward()

                # Update the input audio with gradients
                optimizer.step()

                # Calculating perturbation by subtracting the optimized audio from cloned one
                perturbation = input_audio - input_audio_orig

                # Project the perturbation onto the epsilon ball in range (-eps, eps)
                perturbation = torch.clamp(perturbation, -epsilon, epsilon)

                # Cliping to audio in range (-1, 1)
                input_audio.data = torch.clamp(input_audio_orig + perturbation, -1, 1)

                # Storing model's current inference and target transcription in new variables for computing WER
                string1 = list(filter(lambda x: x!= '',self.INFER(input_audio).split("|")))
                string2 = list(reduce(lambda x,y: x+y, target).split("|"))

                if early_stop:
                    # Computing WER while also making sure length of both strings is same
                    # This will also early stop the attack if we reach our target transcription
                    # before the completion of all iteration
                    if len(string1) == len(string2):
                        if self._wer(string1, string2)[0] == 0:
                            print("Breaking for loop because targeted Attack is performed successfully !")
                            adv_example = input_audio
                            return adv_example.detach().cpu().numpy()

                    elif len(string1) > len(string2):
                        diff = len(string1) - len(string2)
                        for i in range(diff):
                            string2.append("<eps>")
                        if self._wer(string1, string2)[0] == 0:
                            print("Breaking for loop because targeted Attack is performed successfully !")
                            adv_example = input_audio
                            return adv_example.detach().cpu().numpy()

                    else:
                        diff = len(string2) - len(string1)
                        for i in range(diff):
                            string1.append("<eps>")
                        if self._wer(string1, string2)[0] == 0:
                            print("Breaking for loop because targeted Attack is performed successfully !")
                            adv_example = input_audio
                            return adv_example.detach().cpu().numpy()

                elif search_eps:
                    # Computing WER while also making sure length of both strings is same
                    if len(string1) == len(string2):
                        if self._wer(string1, string2)[0] == 0:
                            num_successful_attacks += 1
                            if num_successful_attacks >= num_iter_decrease_eps:
                                successful_attack = True
                                epsilon *= decrease_factor_eps
                                num_successful_attacks = 0
                            else:
                                successful_attack = False

                    elif len(string1) > len(string2):
                        diff = len(string1) - len(string2)
                        for i in range(diff):
                            string2.append("<eps>")
                        if self._wer(string1, string2)[0] == 0:
                            num_successful_attacks += 1
                            if num_successful_attacks >= num_iter_decrease_eps:
                                successful_attack = True
                                epsilon *= decrease_factor_eps
                                num_successful_attacks = 0
                            else:
                                successful_attack = False

                    else:
                        diff = len(string2) - len(string1)
                        for i in range(diff):
                            string1.append("<eps>")
                        if self._wer(string1, string2)[0] == 0:
                            num_successful_attacks += 1
                            if num_successful_attacks >= num_iter_decrease_eps:
                                successful_attack = True
                                epsilon *= decrease_factor_eps
                                num_successful_attacks = 0
                            else:
                                successful_attack = False

            adv_example = input_audio
            return adv_example.detach().cpu().numpy()


        else: # If untargeted

            # Then we will use the model's transcription as our target
            target = self.INFER(input_audio.to(self.device))

            for i in tqdm(range(num_iter), colour="red", leave = leave):

                # We will use the model's transcription as our target
                untarget = self.INFER(input_audio.to(self.device))

                # Encode the target transcription
                encoded_transcription = self._encode_transcription(untarget)

                # Convert the target transcription to a PyTorch tensor
                target_tensor = torch.from_numpy(np.array(encoded_transcription)).to(self.device).long()

                # Zero the gradients
                optimizer.zero_grad()

                # Compute the model’s prediction
                output, _ = self.model(input_audio)

                # Softmax Activation for computing logits
                output = F.log_softmax(output, dim=-1)

                # Compute the CTC loss function
                output_length = torch.tensor([output.shape[1]], dtype=torch.long).to(self.device)
                output = output.transpose(0, 1)
                target_length = torch.tensor([len(encoded_transcription)], dtype=torch.long).to(self.device)
                loss_classifier = -F.ctc_loss(output, target_tensor, output_length, target_length, blank=0, reduction='mean')

                # Regularization term to minimize the perturbation
                loss_norm = torch.norm(input_audio - input_audio_orig)

                # Combine the losses and compute gradients
                loss = (c * loss_norm) + ( loss_classifier)

                # Computing gradients of our input w.r.t loss
                loss.backward()

                # Update the input audio with gradients
                optimizer.step()

                # Calculating perturbation by subtracting the optimized audio from cloned one
                perturbation = input_audio - input_audio_orig

                # Project the perturbation onto the epsilon ball in range (-eps, eps)
                perturbation = torch.clamp(perturbation, -epsilon, epsilon)

                # Cliping to audio in range (-1, 1)
                input_audio.data = torch.clamp(input_audio_orig + perturbation, -1, 1)

                # Storing model's current inference and target transcription in new variables for computing WER
                string1 = list(filter(lambda x: x!= '',self.INFER(input_audio).split("|")))
                string2 = list(reduce(lambda x,y: x+y, target).split("|"))

                if early_stop:
                    # Computing WER while also making sure length of both strings is same
                    # This will also early stop the attack if we reach our target transcription
                    # before the completion of all iteration
                    if len(string1) == len(string2):
                        if self._wer(string1, string2)[0] == 1:
                            print("Breaking for loop because targeted Attack is performed successfully !")
                            adv_example = input_audio
                            return adv_example.detach().cpu().numpy()

                    elif len(string1) > len(string2):
                        diff = len(string1) - len(string2)
                        for i in range(diff):
                            string2.append("<eps>")
                        if self._wer(string1, string2)[0] == 1:
                            print("Breaking for loop because targeted Attack is performed successfully !")
                            adv_example = input_audio
                            return adv_example.detach().cpu().numpy()

                    else:
                        diff = len(string2) - len(string1)
                        for i in range(diff):
                            string1.append("<eps>")
                        if self._wer(string1, string2)[0] == 1:
                            print("Breaking for loop because targeted Attack is performed successfully !")
                            adv_example = input_audio
                            return adv_example.detach().cpu().numpy()

                elif search_eps:
                    # Computing WER while also making sure length of both strings is same
                    if len(string1) == len(string2):
                        if self._wer(string1, string2)[0] == 1:
                            num_successful_attacks += 1
                            if num_successful_attacks >= num_iter_decrease_eps:
                                successful_attack = True
                                epsilon *= decrease_factor_eps
                                num_successful_attacks = 0
                            else:
                                successful_attack = False

                    elif len(string1) > len(string2):
                        diff = len(string1) - len(string2)
                        for i in range(diff):
                            string2.append("<eps>")
                        if self._wer(string1, string2)[0] == 1:
                            num_successful_attacks += 1
                            if num_successful_attacks >= num_iter_decrease_eps:
                                successful_attack = True
                                epsilon *= decrease_factor_eps
                                num_successful_attacks = 0
                            else:
                                successful_attack = False

                    else:
                        diff = len(string2) - len(string1)
                        for i in range(diff):
                            string1.append("<eps>")
                        if self._wer(string1, string2)[0] == 1:
                            num_successful_attacks += 1
                            if num_successful_attacks >= num_iter_decrease_eps:
                                successful_attack = True
                                epsilon *= decrease_factor_eps
                                num_successful_attacks = 0
                            else:
                                successful_attack = False

            adv_example = input_audio
            return adv_example.detach().cpu().numpy()

    def IMPERCEPTIBLE_ATTACK(self, input__: torch.Tensor, target: List[str] = None,
                             epsilon: float = 0.3, c: float = 1e-4, learning_rate1: float = 0.01,
                             learning_rate2: float = 0.01, num_iter1: int = 10000, num_iter2: int = 2000,
                             decrease_factor_eps: float = 1.0, num_iter_decrease_eps: int = 10,
                             optimizer1: str = None, optimizer2: str = None, nested: bool = True ,
                             early_stop_cw: bool = True, search_eps_cw: bool = False, alpha: float = 0.5) -> np.ndarray:

        '''
        Implements the Imperceptible ASR attack, which leverages the strongest white box
        adversarial attack which is CW attack while also masking sure the added perturbation
        is imperceptible to humans using Psychoacoustic Scale. This attack is performed in two
        stages. In first stage we perform simple CW attack and in 2nd stage we make sure our
        added perturbations are imperceptible.
        For more information, see the paper: https://arxiv.org/abs/1903.10346

        INPUT ARGUMENTS:

        input__       : Input audio. Ex: Tensor[0.1,0.3,...] or (samples,)
                        Type: torch.Tensor

        target        : Target transcription (needed if the you want targeted
                        attack) Ex: ["my name is mango."]
                        Type: List[str]
                        CAUTION:
                        Please make sure these characters are also present in the
                        dictionary of the model also

        epsilon       : Noise controlling parameter
                        Type: float

        c             : Regularization term controlling factor
                        Type: float

        learning_rate1: learning_rate of optimizer for stage 1
                        Type: float

        learning_rate2: learning_rate of optimizer for stage 2
                        Type: float

        num_iter1     : Number of iteration of attack stage 1
                        Type: int

        num_iter2     : Number of iteration of attack stage 2
                        Type: int

        decrease_factor_eps   : Factor to decrease epsilon by during optimization
                                Type: float

        num_iter_decrease_eps : Number of iterations after which to decrease epsilon
                                Type: int

        optimizer1     : Name of the optimizer to use for the attack stage 1
                         Type: str

        optimizer2     : Name of the optimizer to use for the attack stage 2
                         Type: str

        nested         : if this attack in being run in a for loop with tqdm
                         Type: bool

        early_stop_cw  : if the user wants 1st stage attack to early stop or not
                         Type: bool

        search_eps_cw  : if the user wants 1st stage attack to search for epsilon
                         value provided a large intial epsilon value is provided
                         Type: bool

        alpha          : regularization term for second stage loss
                         Type: float

        RETURNS:

        np.ndarray     : Perturbed audio
        '''

        input_ = input__.clone()

        # This attack will be targeted for now, therefore...
        assert (target is not None), "Please pass a specific target transcription for performing this attack"

        # checking if the user is running this code in for loop or not
        if nested:
            leave = False

        else:
            leave = True

        #stage 1 of Imperceptible ASR attack
        stageOneAud =  self.CW_ATTACK(input_ , target = target, epsilon = epsilon, c = c, learning_rate = learning_rate1,
                           num_iter = num_iter1, decrease_factor_eps = decrease_factor_eps,
                           num_iter_decrease_eps = num_iter_decrease_eps, optimizer = optimizer1,
                           nested = True, early_stop = early_stop_cw, search_eps = search_eps_cw, targeted = True, internal_call = True)


        # Convert the input audio to a PyTorch tensor
        input_audio = torch.from_numpy(stageOneAud).to(self.device).float()

        # Making audio differentiable
        input_audio.requires_grad_()

        # Cloning the original audio
        input_audio_orig = input_.clone().to(self.device)

        # PSD and threshold calculation
        theta, original_max_psd = self._compute_masking_threshold(input_.numpy().squeeze(), win_length = 2048, hop_length = 512,
                                                                 n_fft = 2048, sample_rate = 16000)

        # Taking transpose of threshold to store it in correct order
        theta = torch.tensor(theta.transpose(1, 0)).to(self.device)

        if optimizer2 == "Adam":

            # Define the optimizer
            optimizer = torch.optim.Adam([input_audio], lr=learning_rate2)

        else:

            # Define the optimizer
            optimizer = torch.optim.SGD([input_audio], lr=learning_rate2)

        # Encode the target transcription
        encoded_transcription = self._encode_transcription(target)

        # Convert the target transcription to a PyTorch tensor
        target_tensor = torch.from_numpy(np.array(encoded_transcription)).to(self.device).long()

        # Preparing relu activation for furthur use
        relu = torch.nn.ReLU()

        losss = []
        examplee = []

        #stage 2 of Imperceptible ASR attack
        for i in tqdm(range(num_iter2), colour = 'red', leave = leave, desc="*"*5+"Attack Stage 2"+"*"*5):

            # Zero the gradients
            optimizer.zero_grad()

            # Compute the model’s prediction
            output, _ = self.model(input_audio)

            # Softmax Activation for computing logits
            output = F.log_softmax(output, dim=-1)

            # Compute the CTC loss function
            output_length = torch.tensor([output.shape[1]], dtype=torch.long).to(self.device)
            output = output.transpose(0, 1)
            target_length = torch.tensor([len(encoded_transcription)], dtype=torch.long).to(self.device)
            loss_classifier = F.ctc_loss(output, target_tensor, output_length, target_length, blank=0, reduction='mean')

            # Regularization term to minimize the perturbation
            loss_regularizer = torch.norm(input_audio - input_audio_orig)

            # Combine the losses and compute gradients
            loss1 = ( torch.tensor(c) * loss_classifier) + (loss_regularizer)

            # Calculating perturbation by subtracting the optimized audio from cloned one
            perturbation = input_audio - input_audio_orig

            # Calculating PSD matrix of perturbation which is added in the clean audio
            psd_transform_delta = self._psd_transform(
                delta=perturbation, original_max_psd=original_max_psd
            )

            # Calculating the loss between perturbation's PSD and original audio's PSD
            loss2 = torch.mean(relu(psd_transform_delta - theta))

            # Summing both CW loss and PSD loss
            loss = loss1.type(torch.float64) + (torch.tensor(alpha).to(self.device) * loss2)

            # Taking mean of both losses
            loss = torch.mean(loss)

            # Computing gradients of our input w.r.t loss
            loss.backward()

            # Update the input audio with gradients
            optimizer.step()

            # Storing model's current inference and target transcription in new variables for computing WER
            string1 = list(filter(lambda x: x!= '',self.INFER(input_audio).split("|")))
            string2 = list(reduce(lambda x,y: x+y, target).split("|"))

            if len(losss) <= 300: #buffer logic

                losss.append(loss)
                examplee.append(input_audio.detach().cpu().numpy())

            else:

                del losss[0]
                del examplee[0]
                losss.append(loss)
                examplee.append(input_audio.detach().cpu().numpy())

            if i % 20 == 0: # if every 20 iterations the transcription matches then alpha value will be increased

                if len(string1) == len(string2):
                    if self._wer(string1, string2)[0] == 0:
                        alpha = alpha * 1.2

                elif len(string1) > len(string2):
                    diff = len(string1) - len(string2)
                    for i in range(diff):
                        string2.append("<eps>")
                    if self._wer(string1, string2)[0] == 0:
                        alpha = alpha * 1.2

                else:
                    diff = len(string2) - len(string1)
                    for i in range(diff):
                        string1.append("<eps>")
                    if self._wer(string1, string2)[0] == 0:
                        alpha = alpha * 1.2

            if i % 50 == 0: # if every 50 iterations the transcription does not match then alpha value will be decreased

                if len(string1) == len(string2):
                    if self._wer(string1, string2)[0] != 0:
                        alpha = alpha * 0.8

                elif len(string1) > len(string2):
                    diff = len(string1) - len(string2)
                    for i in range(diff):
                        string2.append("<eps>")
                    if self._wer(string1, string2)[0] != 0:
                        alpha = alpha * 0.8

                else:
                    diff = len(string2) - len(string1)
                    for i in range(diff):
                        string1.append("<eps>")
                    if self._wer(string1, string2)[0] != 0:
                        alpha = alpha * 0.8

        #return the example with the lowest loss among the stored examples
        minimumLoss = min(losss)
        indexLoss = losss.index(minimumLoss)
        adv_example = examplee[indexLoss]
        return adv_example

    def _psd_transform(self, delta: "torch.Tensor", original_max_psd: np.ndarray) -> "torch.Tensor":

        '''
        Note:
        This code is taken from ART Toolbox by IBM.

        Computes the PSD matrix of the perturbation.

        INPUT ARGUMENTS:

        delta            : It is the perturbation added in the audio.
                           Type: torch.Tensor

        original_max_psd : It is the maximum PDF of the original clean audio.
                           Type: np.ndarray

        RETURNS:

        torch.Tensor : The psd matrix.
        '''

        import torch

        # Get window for the transformation
        window_fn = torch.hann_window  # type: ignore

        # Return STFT of delta
        delta_stft = torch.view_as_real(torch.stft(
          delta,
          n_fft=2048,
          hop_length=512,
          win_length=2048,
          center=False,
          window=window_fn(2048).to(self.device),
          return_complex=True,
        )).to(self.device)

        # Take abs of complex STFT results
        transformed_delta = torch.sqrt(torch.sum(torch.square(delta_stft), -1))

        # Compute the psd matrix
        psd = (8.0 / 3.0) * transformed_delta / 2048
        psd = psd ** 2
        psd = (
          torch.pow(torch.tensor(10.0).type(torch.float64), torch.tensor(9.6).type(torch.float64)).to(
              self.device
          )
          / torch.reshape(torch.tensor(original_max_psd).to(self.device), [-1, 1, 1])
          * psd.type(torch.float64)
        )

        return psd

    def _compute_masking_threshold(self, x: np.ndarray, win_length: int = 2048,
                                  hop_length: int = 512, n_fft: int = 2048,
                                  sample_rate: int = 16000) -> Tuple[np.ndarray, np.ndarray]:

        '''
        Note:
        This code is taken from ART Toolbox by IBM.

        Computes the masking threshold and the maximum psd of the original audio.

        INPUT ARGUMENTS:

        delta            : Original clean audio of shape (seq_length,)
                           Type: np.ndarray

        win_length       : Window length of STFT.
                           Type: int

        hop_length       : hop length of STFT.
                           Type: int

        n_fft            : fft-points argument of STFT.
                           Type: int

        sample_rate      : sample rate of original audio
                           Type: int

        RETURNS:

        Tuple[np.ndarray, np.ndarray] : A tuple containing (masking threshold, maximum psd)
        '''

        # First compute the psd matrix
        # Get window for the transformation
        window = scipy.signal.get_window("hann", win_length, fftbins=True)

        # Do transformation
        transformed_x = librosa.stft(y=x, n_fft=n_fft, hop_length=hop_length, win_length=win_length, window=window, center=False)
        transformed_x *= np.sqrt(8.0 / 3.0)

        psd = abs(transformed_x / win_length)
        original_max_psd = np.max(psd * psd)
        with np.errstate(divide="ignore"):
            psd = (20 * np.log10(psd)).clip(min=-200)
        psd = 96 - np.max(psd) + psd

        # Compute freqs and barks
        freqs = librosa.core.fft_frequencies(sr=sample_rate, n_fft=n_fft)
        barks = 13 * np.arctan(0.00076 * freqs) + 3.5 * np.arctan(pow(freqs / 7500.0, 2))

        # Compute quiet threshold
        ath = np.zeros(len(barks), dtype=np.float64) - np.inf
        bark_idx = int(np.argmax(barks > 1))
        ath[bark_idx:] = (
            3.64 * pow(freqs[bark_idx:] * 0.001, -0.8)
            - 6.5 * np.exp(-0.6 * pow(0.001 * freqs[bark_idx:] - 3.3, 2))
            + 0.001 * pow(0.001 * freqs[bark_idx:], 4)
            - 12
        )

        # Compute the global masking threshold theta
        theta = []

        for i in range(psd.shape[1]):
            # Compute masker index
            masker_idx = scipy.signal.argrelextrema(psd[:, i], np.greater)[0]

            if 0 in masker_idx:
                masker_idx = np.delete(masker_idx, 0)

            if len(psd[:, i]) - 1 in masker_idx:
                masker_idx = np.delete(masker_idx, len(psd[:, i]) - 1)

            barks_psd = np.zeros([len(masker_idx), 3], dtype=np.float64)
            barks_psd[:, 0] = barks[masker_idx]
            barks_psd[:, 1] = 10 * np.log10(
                pow(10, psd[:, i][masker_idx - 1] / 10.0)
                + pow(10, psd[:, i][masker_idx] / 10.0)
                + pow(10, psd[:, i][masker_idx + 1] / 10.0)
            )
            barks_psd[:, 2] = masker_idx

            for j in range(len(masker_idx)):
                if barks_psd.shape[0] <= j + 1:
                    break

                while barks_psd[j + 1, 0] - barks_psd[j, 0] < 0.5:
                    quiet_threshold = (
                        3.64 * pow(freqs[int(barks_psd[j, 2])] * 0.001, -0.8)
                        - 6.5 * np.exp(-0.6 * pow(0.001 * freqs[int(barks_psd[j, 2])] - 3.3, 2))
                        + 0.001 * pow(0.001 * freqs[int(barks_psd[j, 2])], 4)
                        - 12
                    )
                    if barks_psd[j, 1] < quiet_threshold:
                        barks_psd = np.delete(barks_psd, j, axis=0)

                    if barks_psd.shape[0] == j + 1:
                        break

                    if barks_psd[j, 1] < barks_psd[j + 1, 1]:
                        barks_psd = np.delete(barks_psd, j, axis=0)
                    else:
                        barks_psd = np.delete(barks_psd, j + 1, axis=0)

                    if barks_psd.shape[0] == j + 1:
                        break

            # Compute the global masking threshold
            delta = 1 * (-6.025 - 0.275 * barks_psd[:, 0])

            t_s = []

            for m in range(barks_psd.shape[0]):
                d_z = barks - barks_psd[m, 0]
                zero_idx = int(np.argmax(d_z > 0))
                s_f = np.zeros(len(d_z), dtype=np.float64)
                s_f[:zero_idx] = 27 * d_z[:zero_idx]
                s_f[zero_idx:] = (-27 + 0.37 * max(barks_psd[m, 1] - 40, 0)) * d_z[zero_idx:]
                t_s.append(barks_psd[m, 1] + delta[m] + s_f)

            t_s_array = np.array(t_s)

            theta.append(np.sum(pow(10, t_s_array / 10.0), axis=0) + pow(10, ath / 10.0))

        theta_array = np.array(theta)

        return theta_array, original_max_psd

    def _wer(self, reference, prediction) -> Tuple[int, Tuple[int, int, int]]:

        '''
        This method only for use internally that's why I have used underscore before the method.
        This method calculates word error rate of of a single example provided it is given equal length transcription.
        If transcriptions are not equal, make them equal by appending <eps> in which ever transcription who's length is smaller than the other.

        INPUT ARGUMENTS:

        reference     : Ground Truth. Ex ['My', 'name', 'is', 'Bond']
                        Type: List

        prediction    : Model's output transcription. Ex ['My', 'name', 'is', 'Band']
                        Type: List

        RETURNS:

        Tuple[int, Tuple[int, int, int]] : single transcription's WER along with another tuple containing information of (Substitution, Insertion, Deletion)
        '''

        correct = 0
        substitution = 0
        insertion = 0
        deletion = 0
        for i in range(len(reference)):
            if prediction[i] == reference[i]:
                correct +=1
            elif prediction[i] != reference[i] and prediction[i] != '<eps>' and reference[i] != '<eps>':
                substitution+=1
            elif prediction[i] == '<eps>':
                deletion+=1
            elif prediction[i] != reference[i] and reference[i] == '<eps>':
                insertion+=1
        wer = (substitution + insertion + deletion) / (correct + substitution + deletion + insertion)
        return wer, (substitution, insertion, deletion)

    def INFER(self, x: np.ndarray, **kwargs):
        # Put the model in the eval mode
        self.model.eval()

        # Transform x into the model input space
        inputs, _, input_rate, _, _ = self._transform_model_input(x=[x.squeeze().to(self.device)])

        # Compute real input sizes
        input_size = input_rate.mul_(inputs.size()[-1]).int()

        # Call to DeepSpeech model for prediction
        with torch.no_grad():
            outputs, output_size, *_ = self.model(
                inputs.to(self.device), input_size.to(self.device)
            )

        # Check if users want transcription outputs
        transcription_output = kwargs.get("transcription_output", True)

        if transcription_output is False:
            return outputs.cpu().numpy(), output_size.detach().cpu().numpy()

        # Now users want transcription outputs
        # Compute transcription
        decoded_output, _ = self.decoder.decode(
            torch.tensor(outputs.cpu().numpy(), device=self.device), torch.tensor(output_size.detach().cpu().numpy(), device=self.device)
        )
        decoded_output = [do[0] for do in decoded_output]
        decoded_output = np.array(decoded_output)

        self.model.train()

        return decoded_output[0]

    def wer_compute(self, ground_truth: List[str], audios: List[np.ndarray], targeted:bool = False)-> Tuple[int, List[Tuple[int, int ,int]]]:

        '''
        Computes WER of a single audio or batch of audios

        INPUT ARGUMENTS:

        ground_truth : Original transcription or reference transcription.
                        Type: List[str]

        audios       : Audios who's transcription from model will be used.
                        Type: List[np.ndarray]

        targeted     : if the WER should be computed according to the way for
                        targeted attack or not.

        RETURNS:

        Tuple[int, List[Tuple[int, int ,int]]] : average WER of given audio/audios and List of Tuple which contains information of (Substitution, Insertion, and Deletion) of every audio separately passed in a batch
        '''

        wer_count = 0
        sib_saver = []
        if targeted == True:
            for i in range(len(audios)):
                prediction = self.INFER(torch.from_numpy(audios[i]))
                reference  = ground_truth[i].split(" ")
                prediction = list(filter(lambda x: x!='', prediction.split(" ")))
                if len(prediction) == len(reference):
                    word_error_rate = max(1 - self._wer(reference, prediction)[0], 0)
                    sib_saver.append(self._wer(reference, prediction)[1])
                elif len(prediction) > len(reference):
                    diff = len(prediction) - len(reference)
                    for _ in range(diff):
                        reference.append("<eps>")
                    word_error_rate = max(1 - self._wer(reference, prediction)[0], 0)
                    sib_saver.append(self._wer(reference, prediction)[1])
                else:
                    diff = len(reference) - len(prediction)
                    for _ in range(diff):
                        prediction.append("<eps>")
                    word_error_rate = max(1 - self._wer(reference, prediction)[0], 0)
                    sib_saver.append(self._wer(reference, prediction)[1])
                wer_count += word_error_rate
                if i == len(audios) - 1:
                    return wer_count/len(audios), sib_saver
        else:
            for i in range(len(audios)):
                prediction = self.INFER(torch.from_numpy(audios[i]))
                reference  = ground_truth[i].split(" ")
                prediction = list(filter(lambda x: x!='', prediction.split(" ")))
                if len(prediction) == len(reference):
                    word_error_rate = min(self._wer(reference, prediction)[0], 1)
                    sib_saver.append(self._wer(reference, prediction)[1])
                elif len(prediction) > len(reference):
                    diff = len(prediction) - len(reference)
                    for _ in range(diff):
                        reference.append("<eps>")
                    word_error_rate = min(self._wer(reference, prediction)[0], 1)
                    sib_saver.append(self._wer(reference, prediction)[1])
                else:
                    diff = len(reference) - len(prediction)
                    for _ in range(diff):
                        prediction.append("<eps>")
                    word_error_rate = min(self._wer(reference, prediction)[0], 1)
                    sib_saver.append(self._wer(reference, prediction)[1])
                wer_count += word_error_rate
                if i == len(audios) - 1:
                    return wer_count/len(audios), sib_saver

    def set_batchnorm(self, train=False):
      for m in self.model.modules():
          if isinstance(m, torch.nn.modules.batchnorm._BatchNorm):
              m.eval()

In [ ]:
# Loading the audio
input_audio, sample_rate = torchaudio.load('/content/CRDNN_Model/AudioSamplesASR/spk1_snt1.wav')

In [ ]:
# My target
target_transcription = 'WHEN MAN NOTHING KILL IN BIG CAMEL'
true_transcription = 'THE CHILD ALMOST HURT THE SMALL DOG'

In [ ]:
decoder_type = "greedy" # or "beam"

In [208]:
#initializing attack class
attack = ASRAttacks(decoder_type)

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.1.5 to v2.0.4. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../librispeech_pretrained_v3.ckpt`


# <center>**FGSM ATTACK**</center>

```
        Simple fast gradient sign method attack is implemented which is the simplest
        adversarial attack in this domain.
        For more information, see the paper: https://arxiv.org/pdf/1412.6572.pdf

        INPUT ARGUMENTS:

        input__       : Input audio. Ex: Tensor[0.1,0.3,...] or (samples,)
                        Type: torch.Tensor

        target        : Target transcription (needed if the you want targeted
                        attack) Ex: ["my name is mango."].
                        Type: List[str]
                        CAUTION:
                        Please make sure these characters are also present in the
                        dictionary of the model also.

        epsilon       : Noise controlling parameter.
                        Type: float

        targeted      : If the attack should be targeted towards your desired
                        transcription or not.
                        Type: bool
                        CAUTION:
                        Please make to pass your targetted
                        transcription also in this case).

        RETURNS:

        np.ndarray : Perturbed audio
```


# **FGSM TARGETED**

In [ ]:
#FGSM
temp1 = attack.FGSM_ATTACK(input_audio, target_transcription, epsilon = 0.01, targeted = True)

#FGSM PRINT
print(attack.INFER(torch.from_numpy(temp1)))
print(target_transcription)

ILE ON THE AL MONTH EER IOI GAN A
WHEN MAN NOTHING KILL IN BIG CAMEL


For WER use [0] and for in depth information of insertion, selection, and deletion use [1]

In [ ]:
print("Targeted WER is: ", attack.wer_compute([target_transcription], [temp1], targeted= True)[0])

Targeted WER is:  0.0


In [ ]:
info = attack.wer_compute([target_transcription], [temp1], targeted= True)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 2, Substitution: 7, Deletion: 0


In [ ]:
Audio(temp1, rate =16000)

# **FGSM UNTARGETED**

In [ ]:
#FGSM
temp2 = attack.FGSM_ATTACK(input_audio, epsilon = 0.0065, targeted = False)

#FGSM PRINT
print(attack.INFER(torch.from_numpy(temp2)).replace("|"," "))

CHILD ALMOSTE A SMALL DAR


In [ ]:
print("Untargeted WER is: ", attack.wer_compute([target_transcription], [temp2], targeted= False)[0])

Untargeted WER is:  1.0


In [ ]:
info = attack.wer_compute([target_transcription], [temp2], targeted= False)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 5, Deletion: 2


In [ ]:
Audio(temp2, rate =16000)

# <center>**BIM ATTACK**</center>

```
        Basic Itertive Moment attack is implemented which is simple Fast Gradient
        Sign Attack but in loop.
        For more information, see the paper: https://arxiv.org/abs/1607.02533

        INPUT ARGUMENTS:

        input__       : Input audio. Ex: Tensor[0.1,0.3,...] or (samples,)
                        Type: torch.Tensor

        target        : Target transcription (needed if the you want targeted
                        attack) Ex: ["my name is mango."].
                        Type: List[str]
                        CAUTION:
                        Please make sure these characters are also present in the
                        dictionary of the model also.

        epsilon       : Noise controlling parameter.
                        Type: float

        alpha         : Noise contribution in input controlling parameter
                        Type: float

        num_iter      : Number of iteration of attack
                        Type: int

        targeted      : If the attack should be targeted towards your desired
                        transcription or not.
                        Type: bool
                        CAUTION:
                        Please make to pass your targetted
                        transcription also in this case).

        early_stop    : If user wants to stop the attack early if the attack reaches the target transcription before the total number of iterations.
                        Type: bool
                        
        RETURNS:
        
        np.ndarray : Perturbed audio
```


# **BIM TARGETED**

With early stopping:

In [ ]:
#BIM
temp3 = attack.BIM_ATTACK(input_audio, target_transcription, epsilon = 0.0015, alpha = 0.00003,
                   num_iter = 500, targeted = True, early_stop = True)

#BIM PRINT
print(attack.INFER(torch.from_numpy(temp3)))
print(target_transcription)

  0%|          | 0/500 [00:00<?, ?it/s]


 O ION THE AL MON   E AL I GON LON A
WHEN MAN NOTHING KILL IN BIG CAMEL


In [ ]:
print("Targeted WER is: ", attack.wer_compute([target_transcription], [temp3], targeted= True)[0])

Targeted WER is:  0.0


In [ ]:
info = attack.wer_compute([target_transcription], [temp3], targeted= True)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 4, Substitution: 7, Deletion: 0


In [ ]:
Audio(temp3, rate = 16000)

Without early stopping:

In [ ]:
#BIM
temp4 = attack.BIM_ATTACK(input_audio, target_transcription, epsilon = 0.0015, alpha = 0.00009,
                   num_iter = 300, targeted = True, early_stop = False)

#BIM PRINT
print('\n',attack.INFER(torch.from_numpy(temp4)).replace("|"," "))
print(target_transcription)

  0%|          | 0/300 [00:00<?, ?it/s]


 O E OO E E E EE E E E E EN E E E
WHEN MAN NOTHING KILL IN BIG CAMEL


In [ ]:
print("Targeted WER is: ", attack.wer_compute([target_transcription], [temp4], targeted= True)[0])

Targeted WER is:  0.0


In [ ]:
info = attack.wer_compute([target_transcription], [temp4], targeted= True)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 8, Substitution: 7, Deletion: 0


In [ ]:
Audio(temp4, rate = 16000)

# **BIM UNTARGETED**

With early stopping:

In [105]:
#BIM
temp5 = attack.BIM_ATTACK(input_audio, epsilon = 0.0015, alpha = 0.00001,
                   num_iter = 250, targeted = False, early_stop = True)

#BIM PRINT
print('\n',attack.INFER(torch.from_numpy(temp5)))

  0%|          | 0/250 [00:00<?, ?it/s]

Breaking for loop because untargeted Attack is performed successfully !

 THECHILD ALMOST HURT THE SMALL DOG


In [106]:
print("Untargeted WER is: ", attack.wer_compute([target_transcription], [temp5], targeted= False)[0])

Untargeted WER is:  1.0


In [107]:
info = attack.wer_compute([target_transcription], [temp5], targeted= False)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 6, Deletion: 1


In [108]:
Audio(temp5, rate =16000)

Without early stopping:

In [109]:
#BIM
temp6 = attack.BIM_ATTACK(input_audio, epsilon = 0.0015, alpha = 0.00001,
                   num_iter = 250, targeted = False, early_stop = False)

#BIM PRINT
print('\n',attack.INFER(torch.from_numpy(temp6)).replace("|"," "))

  0%|          | 0/250 [00:00<?, ?it/s]


 TE CHILD ANEST HURT THE SMAL DOR


In [110]:
print("Untargeted WER is: ", attack.wer_compute([target_transcription], [temp6], targeted= False)[0])

Untargeted WER is:  1.0


In [111]:
info = attack.wer_compute([target_transcription], [temp6], targeted= False)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 7, Deletion: 0


In [112]:
Audio(temp6, rate =16000)

# <center>**PGD ATTACK**</center>

```
        Projected Gradient Descent attack is implemented which in simple terms is more
        advanced version of BIM. In this attack we project our perturbation back to
        some Lp norm and find perturbations in that particular region.
        For more information, see the paper: https://arxiv.org/abs/1706.06083

        INPUT ARGUMENTS:

        input__       : Input audio. Ex: Tensor[0.1,0.3,...] or (samples,)
                        Type: torch.Tensor

        target        : Target transcription (needed if the you want targeted
                        attack) Ex: ["my name is mango."].
                        Type: List[str]
                        CAUTION:
                        Please make sure these characters are also present in the
                        dictionary of the model also.

        epsilon       : Noise controlling parameter.
                        Type: float

        alpha         : Noise contribution in input controlling parameter
                        Type: float

        num_iter      : Number of iteration of attack
                        Type: int

        targeted      : If the attack should be targeted towards your desired
                        transcription or not.
                        Type: bool
                        CAUTION:
                        Please make to pass your targetted
                        transcription also in this case).

        early_stop    : If user wants to stop the attack early if the attack reaches the target transcription before the total number of iterations.
                        Type: bool
                        
        RETURNS:
        
        np.ndarray : Perturbed audio
```

# **PGD TARGETED**

With early stopping:

In [209]:
#PGD
temp7 = attack.PGD_ATTACK(input_audio, target_transcription, epsilon = 0.0015, alpha = 0.00009,
                   num_iter = 25, targeted = True, early_stop = True)

#PGD PRINT
print(attack.INFER(torch.from_numpy(temp7)))
print(target_transcription)

  0%|          | 0/25 [00:00<?, ?it/s]

Input shape: torch.Size([1, 1, 161, 288])
Max input: tensor([[[-0.3511, -0.3650, -0.3639,  ..., -0.3699, -0.3679, -0.3520],
         [-0.3546, -0.3603, -0.3678,  ..., -0.3567, -0.2838, -0.2175],
         [-0.3694, -0.3646, -0.3473,  ..., -0.2175,  0.0034,  0.2001],
         ...,
         [-0.3726, -0.3716, -0.3655,  ..., -0.3677, -0.3685, -0.3686],
         [-0.3718, -0.3717, -0.3629,  ..., -0.3706, -0.3682, -0.3688],
         [-0.3714, -0.3706, -0.3515,  ..., -0.3731, -0.3681, -0.3709]]])
Delta shape: torch.Size([1, 1, 161, 288])
Max delta: tensor([[[-0.2597, -1.5386, -1.1827,  ..., -0.4188, -0.4406,  1.9095],
         [-0.8599, -1.8824, -1.1335,  ...,  0.4300, -1.0959, -0.4049],
         [-1.7182, -1.3521, -1.1985,  ...,  0.7313,  0.4506, -1.5462],
         ...,
         [-1.4775, -0.7896,  2.5528,  ...,  1.6173,  0.0621,  1.7142],
         [ 0.7199,  0.6520,  1.0992,  ...,  0.4784,  0.8264,  1.9880],
         [ 0.1240,  1.0881, -1.7787,  ..., -0.7813, -1.1658,  0.4831]]],
       gra

TypeError: ignored

In [120]:
print("Targeted WER is: ", attack.wer_compute([target_transcription], [temp7], targeted= True)[0])

Targeted WER is:  0.0


In [121]:
info = attack.wer_compute([target_transcription], [temp7], targeted= True)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 7, Deletion: 0


In [148]:
Audio(temp7, rate =16000)

Without early stopping:

In [124]:
#PGD
temp8 = attack.PGD_ATTACK(input_audio, target_transcription, epsilon = 0.0015, alpha = 0.00009,
                   num_iter = 250, targeted = True, early_stop = False)

#PGD PRINT
print('\n',attack.INFER(torch.from_numpy(temp8)).replace("|"," "))
print(target_transcription)

  0%|          | 0/250 [00:00<?, ?it/s]


 THE CHILD ALMOST HURT THE SMALL DOG
WHEN MAN NOTHING KILL IN BIG CAMEL


In [125]:
print("Targeted WER is: ", attack.wer_compute([target_transcription], [temp8], targeted= True)[0])

Targeted WER is:  0.0


In [126]:
info = attack.wer_compute([target_transcription], [temp8], targeted= True)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 7, Deletion: 0


In [127]:
Audio(temp8, rate =16000)

# **PGD UNTARGETED**

With early stopping:

In [163]:
#PGD
temp9 = attack.PGD_ATTACK(input_audio, epsilon = 0.15, alpha = 0.01,
                   num_iter = 100, targeted = False, early_stop = True)

#PGD PRINT
print(attack.INFER(torch.from_numpy(temp9)))

  0%|          | 0/100 [00:00<?, ?it/s]

In [164]:
print(attack.INFER(torch.from_numpy(temp9)))

In [165]:
print("Untargeted WER is: ", attack.wer_compute([target_transcription], [temp9], targeted= False)[0])

Untargeted WER is:  1.0


In [166]:
info = attack.wer_compute([target_transcription], [temp9], targeted= False)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 0, Deletion: 7


In [167]:
Audio(temp9, rate =16000)

Without early stopping:

In [169]:
#PGD
temp10 = attack.PGD_ATTACK(input_audio, epsilon = 0.0015, alpha = 0.0001,
                   num_iter = 100, targeted = False, early_stop = True)

#PGD PRINT
print(attack.INFER(torch.from_numpy(temp10)))

  0%|          | 0/100 [00:00<?, ?it/s]

In [170]:
print("Untargeted WER is: ", attack.wer_compute([target_transcription], [temp10], targeted= False)[0])

Untargeted WER is:  1.0


In [171]:
info = attack.wer_compute([target_transcription], [temp10], targeted= False)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 0, Deletion: 7


In [172]:
Audio(temp10, rate =16000)

# <center>**CW ATTACK**</center>

```
        Implements the Carlini and Wagner attack, the strongest white box
        adversarial attack. This attack uses an optimization strategy to find the
        adversarial transcription while keeping the perturbation as low as possible.
        For more information, see the paper: https://arxiv.org/pdf/1801.01944.pdf
        
        INPUT ARGUMENTS:
        
        input__       : Input audio. Ex: Tensor[0.1,0.3,...] or (samples,)
                        Type: torch.Tensor
                        
        target        : Target transcription (needed if the you want targeted
                        attack) Ex: ["my name is mango."].
                        Type: List[str]
                        CAUTION:
                        Please make sure these characters are also present in the
                        dictionary of the model also.
                        
        epsilon       : Noise controlling parameter.
                        Type: float
                        
        c             : Regularization term controlling factor.
                        Type: float
                        
        learning_rate : learning_rate of optimizer.
                        Type: float
                        
        num_iter      : Number of iteration of attack.
                        Type: int
                        
        decrease_factor_eps   : Factor to decrease epsilon during search
                                Type: float
                                
        num_iter_decrease_eps : Number of iterations after which to decrease epsilon
                                Type: int
                                
        optimizer     : Name of the optimizer to use for the attack.
                        Type: str
                        
        nested        : if this attack in being run in a for loop with tqdm
                        Type: bool
                        
        early_stop    : if the user wants to end the attack as soon as the attack
                        gets the target transcription.
                        Type: bool
                        
        search_eps    : if the user wants the attack to search for the epsilon value
                        on its own provided the initial epsilon value of large.
                        Type: bool
        
        targeted      : If the attack should be targeted towards your desired
                        transcription or not.
                        Type: bool
                        CAUTION:
                        Please make to pass your targetted
                        transcription also in this case).
                        
        RETURNS:
        
        np.ndarray : Perturbed audio
```

# **CW TARGETED**

With early stopping:

In [ ]:
#CW
temp11 = attack.CW_ATTACK(input_audio, target, epsilon = 0.0015, c = 10,
                  learning_rate = 0.00001, num_iter = 10000, decrease_factor_eps = 1,
                  num_iter_decrease_eps = 10, optimizer = None, nested = True,
                  early_stop = True, search_eps = False, targeted = True)

#CW PRINT
print('\n',attack.INFER(torch.from_numpy(temp11)).replace("|"," "))
print(target_transcription)

  0%|          | 0/10000 [00:00<?, ?it/s]


 CHIL MAN NOTHING KILL IN BIG CAMEL
WHEN MAN NOTHING KILL IN BIG CAMEL


In [ ]:
print("Targeted WER is: ", attack.wer_compute([target_transcription], [temp11], targeted= True)[0])

Targeted WER is:  0.8571428571428572


In [ ]:
info = attack.wer_compute([target_transcription], [temp11], targeted= True)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 1, Deletion: 0


In [ ]:
Audio(temp11, rate =16000)

Without early stopping:

In [ ]:
#CW
temp12 = attack.CW_ATTACK(input_audio, target, epsilon = 0.0015, c = 10,
                  learning_rate = 0.00001, num_iter = 10000, decrease_factor_eps = 1,
                  num_iter_decrease_eps = 10, optimizer = None, nested = True,
                  early_stop = False, search_eps = False, targeted = True)

#CW PRINT
print('\n',attack.INFER(torch.from_numpy(temp12)).replace("|"," "))
print(target_transcription)

  0%|          | 0/10000 [00:00<?, ?it/s]


 CHIL MAN NOTHING KILL IN BIG CAMEL
WHEN MAN NOTHING KILL IN BIG CAMEL


In [ ]:
print("Targeted WER is: ", attack.wer_compute([target_transcription], [temp12], targeted= True)[0])

Targeted WER is:  0.8571428571428572


In [ ]:
info = attack.wer_compute([target_transcription], [temp12], targeted= True)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 1, Deletion: 0


In [ ]:
Audio(temp12, rate =16000)

# **CW UNTARGETED**

With early stopping:

In [ ]:
#CW
temp13 = attack.CW_ATTACK(input_audio, epsilon = 0.0015, c = 10,
                  learning_rate = 0.00001, num_iter = 10000, decrease_factor_eps = 1,
                  num_iter_decrease_eps = 10, optimizer = None, nested = True,
                  early_stop = True, search_eps = False, targeted = False)

#CW PRINT
print('\n',attack.INFER(torch.from_numpy(temp13)).replace("|"," "))

  0%|          | 0/10000 [00:00<?, ?it/s]

Breaking for loop because targeted Attack is performed successfully !

 THECHILD ALMOST HURT HE SMALL DOG  


In [ ]:
print("Untargeted WER is: ", attack.wer_compute([target_transcription], [temp13], targeted= False)[0])

Untargeted WER is:  1.0


In [ ]:
info = attack.wer_compute([target_transcription], [temp13], targeted= False)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 6, Deletion: 1


In [ ]:
Audio(temp13, rate =16000)

Without early stopping:

In [ ]:
#CW
temp14 = attack.CW_ATTACK(input_audio, epsilon = 0.0015, c = 10,
                  learning_rate = 0.00001, num_iter = 10000, decrease_factor_eps = 1,
                  num_iter_decrease_eps = 10, optimizer = None, nested = True,
                  early_stop = False, search_eps = False, targeted = False)

#CW PRINT
print('\n',attack.INFER(torch.from_numpy(temp14)).replace("|"," "))

  0%|          | 0/10000 [00:00<?, ?it/s]


  EY E EIELEWE ONAOAWLAMONWASTIEOE EOIKIKYEIOWO I TINM LOENMEDO NOENOEKE


In [ ]:
print("Untargeted WER is: ", attack.wer_compute([target_transcription], [temp14], targeted= False)[0])

Untargeted WER is:  1.0


In [ ]:
info = attack.wer_compute([target_transcription], [temp14], targeted= False)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 2, Substitution: 7, Deletion: 0


In [ ]:
Audio(temp14, rate = 16000)

# <center>**IMPERCEPTIBLE ASR ATTACK**</center>


```
        Implements the Imperceptible ASR attack, which leverages the strongest white box
        adversarial attack which is CW attack while also masking sure the added perturbation
        is imperceptible to humans using Psychoacoustic Scale. This attack is performed in two
        stages. In first stage we perform simple CW attack and in 2nd stage we make sure our
        added perturbations are imperceptible.
        For more information, see the paper: https://arxiv.org/abs/1903.10346
        
        INPUT ARGUMENTS:
        
        input__       : Input audio. Ex: Tensor[0.1,0.3,...] or (samples,)
                        Type: torch.Tensor
                        
        target        : Target transcription (needed if the you want targeted
                        attack) Ex: ["my name is mango."]
                        Type: List[str]
                        CAUTION:
                        Please make sure these characters are also present in the
                        dictionary of the model also
                        
        epsilon       : Noise controlling parameter
                        Type: float
                        
        c             : Regularization term controlling factor
                        Type: float
                        
        learning_rate1: learning_rate of optimizer for stage 1
                        Type: float
        
        learning_rate2: learning_rate of optimizer for stage 2
                        Type: float
                        
        num_iter1     : Number of iteration of attack stage 1
                        Type: int
        
        num_iter2     : Number of iteration of attack stage 2
                        Type: int
                        
        decrease_factor_eps   : Factor to decrease epsilon by during optimization
                                Type: float
                                
        num_iter_decrease_eps : Number of iterations after which to decrease epsilon
                                Type: int
                                
        optimizer1     : Name of the optimizer to use for the attack stage 1
                         Type: str
                         
        optimizer2     : Name of the optimizer to use for the attack stage 2
                         Type: str
                         
        nested         : if this attack in being run in a for loop with tqdm
                         Type: bool
        
        early_stop_cw  : if the user wants 1st stage attack to early stop or not
                         Type: bool
        
        search_eps_cw  : if the user wants 1st stage attack to search for epsilon
                         value provided a large intial epsilon value is provided
                         Type: bool
                         
        alpha          : regularization term for second stage loss
                         Type: float
                         
        RETURNS:
        
        np.ndarray     : Perturbed audio
```

With early stopping:

In [ ]:
temp15 = attack.IMPERCEPTIBLE_ATTACK(torch.nn.functional.pad(input_audio, (0, 1000)), target, epsilon = 0.015, c = 10, learning_rate1 = 0.001,
                                    learning_rate2 = 0.0001, num_iter1 = 1000, num_iter2 = 15000, decrease_factor_eps = 1,
                                    num_iter_decrease_eps = 10, optimizer1 = None, optimizer2 = "Adam",nested = True ,
                                    early_stop_cw = True, search_eps_cw = False, alpha = 0.05)

# Attack's transcription
print('\n',attack.INFER(torch.from_numpy(temp15)).replace("|"," "))

***** Attack Stage 1 *****


  0%|          | 0/1000 [00:00<?, ?it/s]

Breaking for loop because targeted Attack is performed successfully !
***** Attack Stage 2 *****


  0%|          | 0/15000 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/functional.py:641: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at ../aten/src/ATen/native/SpectralOps.cpp:862.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]



 WHEN MAN NOTHING KILL IN BIG CAMEL


In [ ]:
print("Targeted WER is: ", attack.wer_compute([target_transcription], [temp15], targeted= True)[0])

Targeted WER is:  1.0


In [ ]:
info = attack.wer_compute([target_transcription], [temp15], targeted= True)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 0, Deletion: 0


In [ ]:
Audio(temp15[:,:-1000], rate = 16000)

Without early stopping:

In [ ]:
temp16 = attack.IMPERCEPTIBLE_ATTACK(torch.nn.functional.pad(input_audio, (0, 1000)), target, epsilon = 0.015, c = 10, learning_rate1 = 0.001,
                                    learning_rate2 = 0.0001, num_iter1 = 1000, num_iter2 = 15000, decrease_factor_eps = 1,
                                    num_iter_decrease_eps = 10, optimizer1 = None, optimizer2 = "Adam",nested = True ,
                                    early_stop_cw = False, search_eps_cw = False, alpha = 0.05)

# Attack's transcription
print('\n',attack.INFER(torch.from_numpy(temp16)).replace("|"," "))

***** Attack Stage 1 *****


  0%|          | 0/1000 [00:00<?, ?it/s]

***** Attack Stage 2 *****


  0%|          | 0/15000 [00:00<?, ?it/s]

In [ ]:
print("Targeted WER is: ", attack.wer_compute([target_transcription], [temp16], targeted= True)[0])

Targeted WER is:  0.0


In [ ]:
info = attack.wer_compute([target_transcription], [temp16], targeted= True)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 0, Deletion: 7


In [ ]:
Audio(temp16[:,:-1000], rate = 16000)

----------------------------

# <center> **COMPARISON**

METRIC I USED FOR COMPARISON

In [ ]:
# SNR metric for comparison to check which attack adds less noise
def Metricsnr(original, noisy):
    original_power = 20 * np.log10(np.mean(original ** 2))
    noise_power = 20 * np.log10(np.mean((original - noisy) ** 2))
    snr = noise_power - original_power
    return snr

In [ ]:
orig = input_audio.numpy() # clean audio
orig.shape

(1, 45920)

-------------------

# **TARGETED ATTACKS**

With early stopping

In [ ]:
Audio(temp3, rate = 16000) #BIM targeted

In [ ]:
Audio(temp7, rate = 16000) #PGD targeted

In [ ]:
Audio(temp11, rate = 16000) #CW targeted

In [ ]:
Audio(temp15[:,:-1000], rate = 16000) # Imperceptible ASR

In [ ]:
print("LOWER SNR IS BETTER OVER BECAUSE OF THE WAY IT IS MEASURED")
print("BIM SNR                  :",Metricsnr(orig, temp3))
print("PGD SNR                  :",Metricsnr(orig, temp7))
print("CW SNR                   :",Metricsnr(orig, temp11))
print("IMPERCEPTIBLE ATTACK SNR :",Metricsnr(orig, temp15[:,:-1000]))

LOWER SNR IS BETTER OVER BECAUSE OF THE WAY IT IS MEASURED
BIM SNR                  : -41.37290000915527
PGD SNR                  : -52.61950492858887
CW SNR                   : -56.00250244140625
IMPERCEPTIBLE ATTACK SNR : -34.916744232177734


Without early stopping

In [ ]:
Audio(temp4, rate = 16000) #BIM targeted

In [ ]:
Audio(temp8, rate = 16000) #PGD targeted

In [ ]:
Audio(temp12, rate = 16000) #CW targeted

In [ ]:
Audio(temp16[:,:-1000], rate = 16000) # Imperceptible ASR

In [ ]:
print("LOWER SNR IS BETTER OVER BECAUSE OF THE WAY IT IS MEASURED")
print("BIM SNR                  :",Metricsnr(orig, temp4))
print("PGD SNR                  :",Metricsnr(orig, temp8))
print("CW SNR                   :",Metricsnr(orig, temp12))
print("IMPERCEPTIBLE ATTACK SNR :",Metricsnr(orig, temp16[:,:-1000]))

LOWER SNR IS BETTER OVER BECAUSE OF THE WAY IT IS MEASURED
BIM SNR                  : -23.13732147216797
PGD SNR                  : -52.24007606506348
CW SNR                   : -55.575008392333984
IMPERCEPTIBLE ATTACK SNR : nan


-------------------

-------------------

# **UNTARGETED ATTACKS**

With early stopping

In [ ]:
Audio(temp5, rate = 16000) #BIM untargeted

In [ ]:
Audio(temp9, rate = 16000) #PGD untargeted

In [ ]:
Audio(temp13, rate = 16000) #CW untargeted

In [ ]:
print("LOWER SNR IS BETTER OVER BECAUSE OF THE WAY IT IS MEASURED")
print("BIM SNR:",Metricsnr(orig, temp5))
print("PGD SNR:",Metricsnr(orig, temp9))
print("CW SNR :",Metricsnr(orig, temp13))

LOWER SNR IS BETTER OVER BECAUSE OF THE WAY IT IS MEASURED
BIM SNR: -79.11591529846191
PGD SNR: -52.33769416809082
CW SNR : -92.73252487182617


Without early stopping

In [ ]:
Audio(temp6, rate = 16000) #BIM untargeted

In [ ]:
Audio(temp10, rate = 16000) #PGD untargeted

In [ ]:
Audio(temp14, rate = 16000) #PGD untargeted

In [ ]:
print("LOWER SNR IS BETTER OVER BECAUSE OF THE WAY IT IS MEASURED")
print("BIM SNR:",Metricsnr(orig, temp6))
print("PGD SNR:",Metricsnr(orig, temp10))
print("CW SNR :",Metricsnr(orig, temp14))

LOWER SNR IS BETTER OVER BECAUSE OF THE WAY IT IS MEASURED
BIM SNR: -23.9810848236084
PGD SNR: -52.50859260559082
CW SNR : -55.923452377319336


-------------------